In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        # - "03_27_24_RandomCircles_MassiveEnsemble_CE"
        # - "03_28_24_RandomCircles_MassiveEnsemble_Dice"
        # - "03_31_24_RandomCircles_MassiveEnsemble_Hausdorff"
        - "03_31_24_RandomCircles_AdditionalData_MassiveEnsemble"
    
# log_attributes:
#     03_26_24_RandomCircles_MixedEnsembleStrategies:
#         loss_func: "CE"

calibration:
    metric_cfg_file: "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference/Calibration_Metrics.yaml"

options:
    add_baseline_rows: True  
    load_pixel_meters: False 
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    load_groupavg_metrics: False
    remove_shared_columns: False
    equal_rows_per_cfg_assert: True 

In [ ]:
from ese.experiment.analysis.analyze_inf import load_cal_inference_stats

image_info_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False,
)

In [ ]:
image_info_df['calibrator'].unique()

In [ ]:
# Sort the image_info_df by method name, so everything appears nicely
image_info_df = image_info_df.sort_values(by=['method_name', 'calibrator'])
# Make sure that the model_class 'Uncalibrated' is first
image_info_df['calibrator'] = image_info_df['calibrator'].astype('category')
image_info_df['calibrator'] = image_info_df['calibrator'].cat.reorder_categories([
    'None',
    'Uncalibrated',
    # 'TempScaling', 
    # 'LTS'
])

In [ ]:
# Select only the rows corresponding to group methods
image_info_df = image_info_df[image_info_df['model_type'] == 'group']

In [ ]:
image_info_df['method_name'].unique()

In [ ]:
image_info_df['method_name'] = image_info_df['method_name'].astype('category')
image_info_df['method_name'] = image_info_df['method_name'].cat.reorder_categories([
    'Average UNet',
    'Ensemble (mean, probs)',
    'Ensemble (max, probs)',
])

image_info_df['split'] = image_info_df['split'].astype('category')
image_info_df['split'] = image_info_df['split'].cat.reorder_categories([
    'val',
    'cal'
])

## ECE Metrics

In [ ]:
g = sns.relplot(
    data=image_info_df,
    x="num_ensemble_members",
    y="ECE",
    hue="calibrator",
    style="method_name",
    col="split",
    kind="line",
    height=9,
)
g.fig.suptitle("ECE by Calibration Method and Calibration Loss Function", fontsize=25)
g.set_xlabels("# Ensemble Members")
g.fig.subplots_adjust(top=0.85)
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)

In [ ]:
def group_by_config(in_df):
    grouped_df = in_df.groupby([
        'ensemble_hash',
        'method_name',
        'calibrator',
        'split',
        'num_ensemble_members',
        'image_metric'
    ])
    # Mean over the metric_score columns
    meaned_groups =  grouped_df.agg({'metric_score': 'mean'}).reset_index()
    # Drop the NaN rows
    grouped_cfg = meaned_groups.dropna().reset_index(drop=True)
    return grouped_cfg

In [ ]:
image_info_df['image_metric'].unique()

In [ ]:
ece_metric_df = group_by_config(image_info_df[image_info_df['image_metric'] == 'Image_ECE'])

g = sns.relplot(
    data=ece_metric_df,
    x="num_ensemble_members",
    y="metric_score",
    hue="calibrator",
    style="method_name",
    kind="line",
    col="split",
    height=9,
)
# Set the title of the bar plot
g.fig.suptitle("Image-level ECE for Different Calibration Methods and ", fontsize=25)
g.fig.subplots_adjust(top=0.85)
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)
# Change the y axis label to say ECEkjkj
g.set_ylabels("Image-level ECE")
g.set_xlabels("# Ensemble Members")
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))

In [ ]:
dice_metric_df = group_by_config(image_info_df[image_info_df['image_metric'] == 'Dice'])

g = sns.relplot(
    data=dice_metric_df,
    x="num_ensemble_members",
    y="metric_score",
    hue="calibrator",
    style="method_name",
    kind="line",
    col="split",
    height=9,
)
# Set the title of the bar plot
g.fig.suptitle("Dice for Different Calibration Methods", fontsize=25)
g.fig.subplots_adjust(top=0.85)
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)
# Change the y axis label
g.set_ylabels("Dice")
g.set_xlabels("# Ensemble Members")
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))

In [ ]:
hd_metric_df = group_by_config(image_info_df[image_info_df['image_metric'] == 'HD95'])

g = sns.relplot(
    data=hd_metric_df,
    x="num_ensemble_members",
    y="metric_score",
    hue="calibrator",
    style="method_name",
    kind="line",
    col="split",
    height=9,
)
# Set the title of the bar plot
g.fig.suptitle("HD95 for Different Calibration Methods", fontsize=25)
g.fig.subplots_adjust(top=0.85)
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)
# Change the y axis label
g.set_ylabels("HD95")
g.set_xlabels("# Ensemble Members")
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))

In [ ]:
boundary_metric_df = group_by_config(image_info_df[image_info_df['image_metric'] == 'BoundaryIOU'])

g = sns.relplot(
    data=boundary_metric_df,
    x="num_ensemble_members",
    y="metric_score",
    hue="calibrator",
    style="method_name",
    kind="line",
    col="split",
    height=9,
)
# Set the title of the bar plot
g.fig.suptitle("Boundary IoU for Different Calibration Methods", fontsize=25)
g.fig.subplots_adjust(top=0.85)
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)
# Change the y axis label
g.set_ylabels("Boundary IoU")
g.set_xlabels("# Ensemble Members")
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))

In [ ]:
boundary_metric_df = group_by_config(image_info_df[image_info_df['image_metric'] == 'Accuracy'])

g = sns.relplot(
    data=boundary_metric_df,
    x="num_ensemble_members",
    y="metric_score",
    hue="calibrator",
    style="method_name",
    kind="line",
    col="split",
    height=9,
)
# Set the title of the bar plot
g.fig.suptitle("Pixel-Accuracy for Different Calibration Methods", fontsize=25)
g.fig.subplots_adjust(top=0.85)
# Separate the columns slightly
g.fig.subplots_adjust(wspace=0.07)
# Change the y axis label
g.set_ylabels("Pixel-Accuracy")
g.set_xlabels("# Ensemble Members")
g.set(xticks=[2, 4, 8, 16, 32])
g.set(xlim=(2, 32))